In [1]:
from gym_model import Gym
from mesa.experimental import JupyterViz

In [2]:
gym = Gym(1)
gym.step()

In [2]:
page = JupyterViz(
    Gym,
    model_params={"num_trainees": 1},
    measures=[],
    name="Test gym",
    agent_portrayal=lambda agent: agent.portrayal
)

page

Cannot show ipywidgets in text

free space from (0, 0): [(0, 1)]
free space from (0, 1): [(0, 0), (0, 2)]
free space from (0, 2): [(0, 1), (0, 3)]
free space from (0, 3): [(0, 2)]
free space from (0, 2): [(0, 1), (0, 3)]
